In [2]:
import numpy as np
import cv2

In [17]:
## 트랙바 (이진화, 어파인변환)
def onChange(pos): ## 콜백 함수
    pass

src_gray = cv2.imread('image/cb.jpg', 0) ## 그레이스케일
src_color = cv2.imread('image/cb.jpg', 1) ## bgr 컬러 스케일
src_gray = cv2.resize(src_gray,dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)
src_color = cv2.resize(src_color,dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)
height, width, channel = src_color.shape
cv2.namedWindow("Track_threshold") ## 스레스홀드를 사용하기위한 윈도우 창
cv2.namedWindow("Track_warpAffine") ## 어파인변환을 사용하기 위한 윈도우 창
cv2.createTrackbar('threshold', 'Track_threshold', 0, 255, onChange)
cv2.createTrackbar('maxValue', 'Track_threshold', 0, 255 , onChange)
cv2.createTrackbar('rotation', 'Track_warpAffine', 0, 360 , onChange)

cv2.setTrackbarPos('threshold','Track_threshold',127)
cv2.setTrackbarPos('maxValue','Track_threshold',255)
cv2.setTrackbarPos('rotation','Track_warpAffine',180)

while cv2.waitKey(1) != ord('q') : ## q 를 누르면 종료
    
    thresh = cv2.getTrackbarPos('threshold','Track_threshold')
    maxval = cv2.getTrackbarPos('maxValue','Track_threshold')
    _,binary = cv2.threshold(src_gray, thresh, maxval, cv2.THRESH_BINARY)
    
    rot = cv2.getTrackbarPos('rotation','Track_warpAffine')   
    matrix = cv2.getRotationMatrix2D((width/2,height/2), rot, 1)
    warpAffine = cv2.warpAffine(src_color, matrix, (width,height), borderMode = cv2.BORDER_DEFAULT)
    
    cv2.imshow('Track_threshold',binary)
    cv2.imshow('Track_warpAffine',warpAffine)
    
cv2.destroyAllWindows()
    

In [18]:
## 트랙바 (blur, hsv 컬러 추출)
def onChange(pos) : ## 콜백 함수 
    pass
src = cv2.imread('image/tomato.jpg',1)
src = cv2.resize(src, dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
cv2.namedWindow("Track_blur") ## blur 윈도우 창
cv2.namedWindow("Track_hsv")  ## hsv 윈도우 창

cv2.createTrackbar('ksize_x', 'Track_blur', 1, 20, onChange) ## 커널 가로 축
cv2.createTrackbar('ksize_y', 'Track_blur', 1, 20, onChange) ## 커널 세로 축
cv2.createTrackbar('h_min', 'Track_hsv', 0, 180, onChange)   ## h의 최대값
cv2.createTrackbar('h_max', 'Track_hsv', 0, 180, onChange)   ## h의 최소값

cv2.setTrackbarPos('ksize_x', 'Track_blur', 1)
cv2.setTrackbarPos('ksize_y', 'Track_blur', 1)
cv2.setTrackbarMin('ksize_x', 'Track_blur', 1) ## 커널 사이즈가 0일 수 없기 때문에 트랙바 변수 최솟값을 1로 설정
cv2.setTrackbarMin('ksize_y', 'Track_blur', 1) ## 커널 사이즈가 0일 수 없기 때문에 트랙바 변수 최솟값을 1로 설정
cv2.setTrackbarPos('h_min', 'Track_hsv', 8)
cv2.setTrackbarPos('h_max', 'Track_hsv', 20)

while cv2.waitKey(1) != ord('q') :  ## q를 누르면 종료
    x = cv2.getTrackbarPos('ksize_x', 'Track_blur')
    y = cv2.getTrackbarPos('ksize_y', 'Track_blur')
    blur = cv2.blur(src, (x,y), anchor = (-1,-1))
    h_min = cv2.getTrackbarPos('h_min', 'Track_hsv')
    h_max = cv2.getTrackbarPos('h_max', 'Track_hsv')
    h, s, v = cv2.split(hsv)
    h = cv2.inRange(h, h_min, h_max)
    color = cv2.bitwise_and(hsv,hsv, mask = h)
    color = cv2.cvtColor(color, cv2.COLOR_HSV2BGR)
    cv2.imshow("Track_blur", blur)
    cv2.imshow('Track_hsv', color)
cv2.destroyAllWindows()

In [27]:
## 허프변환 원 검출

src = cv2.imread('image/coin.jpg', 1)
dst = src.copy()
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)


circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 50, param1 = 150, param2 = 40, minRadius = 10, maxRadius = 50)

for i in circles[0]:
    cv2.circle(dst,(i[0].astype(np.uint16),i[1].astype(np.uint16)),i[2].astype(np.uint16), (0,0,0), 5)
    
    
cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
## 템플릿 검출

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
templit = cv2.imread("image/sky.jpg", 0)


while capture.isOpened():
    ret, frame = capture.read()
    if not ret:
        break
        
    if cv2. waitKey(33) == ord('q'):
        break
        
    
    dst = frame.copy()
    dst = cv2.cvtColor(dst,cv2.COLOR_BGR2GRAY)
    result = cv2.matchTemplate(dst, templit, cv2.TM_CCOEFF_NORMED)
    
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
    
    x, y = maxLoc
    h, w = templit.shape
    frame = cv2.rectangle(frame, (x, y), (x +  w, y + h) , (0, 0, 255), 1)
        
    
    cv2.imshow('Video', frame)
    
    
capture.release()    
cv2.destroyAllWindows()

In [ ]:
## ORB 알고리즘 (fast detector(검출기), brief descriptor(기술자))


src = cv2.imread('image/apple_book.jpg', 1)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
target = cv2.imread('image/apple,jpg', 0)

orb = cv2.ORB_create(
    nfeatures = 40000,
    scaleFactor = 1.2,
    nlevels = 8,
    edgeThreshold = 31,
    firstLevel = 0,
    WTA_K = 2,
    scorType = cv2.ORB_HARRIS_SCORE,
    patchSize = 31,
    fastThreshold = 20,
    
)

kp1, des1 = orb.detectAndCompute(gray, None)
kp2, des2 = orb.detectAndCompute(target, None)